# 2. Explore stations data by specific time

In [72]:
import os
import sys
import json
import random
import requests
import plotly.graph_objects as go

from time import sleep
from tqdm import tqdm
from redis import Redis
from dotenv import load_dotenv

sys.path.append("/Users/alexeylukyanov/Projects/WeatherData/src")
from weathergov.constants import APP_NAME
from weathergov.utils.redis_utils import RedisInfo

### Constants

In [3]:
load_dotenv()

True

In [4]:
weather_gov_api_url = os.environ.get('WEATHER_GOV_API_URL')

In [5]:
weather_gov_api_url

'https://api.weather.gov'

In [18]:
headers = {
    "User-Agent": APP_NAME
}

In [8]:
ri = RedisInfo.load()

In [9]:
ri.host

'localhost'

In [13]:
rc = Redis(
    host=ri.host,
    port=ri.port,
    password=ri.password,
    decode_responses=True
)

### Get all the station IDs

In [53]:
stations_ids = rc.smembers("weather_station:weather.gov:station_ids")

In [54]:
stations_ids = list(stations_ids)

In [56]:
len(stations_ids)

46483

In [59]:
random.choice(stations_ids)

'COOPCSAT2'

In [30]:
obs_time = "2024-07-11T22:59:00+00:00"

In [31]:
response = requests.get(f"{weather_gov_api_url}/stations/{random.choice(stations_ids)}/observations/{obs_time}", headers=headers)

In [32]:
response.json()

{'correlationId': '6ffcdf11',
 'title': 'Not Found',
 'type': 'https://api.weather.gov/problems/NotFound',
 'status': 404,
 'detail': 'Not Found',
 'instance': 'https://api.weather.gov/requests/6ffcdf11'}

In [60]:
station_id = random.choice(stations_ids)

In [61]:
station_id

'TS654'

In [62]:
response = requests.get(f"{weather_gov_api_url}/stations/{station_id}/observations", headers=headers)

In [63]:
rj = response.json()

In [52]:
rc.sismember("weather_station:weather.gov:station_ids", "LAIG7")

0

In [64]:
rj.keys()

dict_keys(['@context', 'type', 'features'])

In [66]:
rj['features'][0]['properties']['timestamp']

'2024-07-12T14:53:00+00:00'

In [67]:
rj['features'][0]['properties']['temperature']['value']

23.89

In [68]:
rj['features'][0]['properties']

{'@id': 'https://api.weather.gov/stations/TS654/observations/2024-07-12T14:53:00+00:00',
 '@type': 'wx:ObservationStation',
 'elevation': {'unitCode': 'wmoUnit:m', 'value': 2048.3},
 'station': 'https://api.weather.gov/stations/TS654',
 'timestamp': '2024-07-12T14:53:00+00:00',
 'rawMessage': '',
 'textDescription': '',
 'icon': None,
 'presentWeather': [],
 'temperature': {'unitCode': 'wmoUnit:degC',
  'value': 23.89,
  'qualityControl': 'V'},
 'dewpoint': {'unitCode': 'wmoUnit:degC',
  'value': 0.23,
  'qualityControl': 'V'},
 'windDirection': {'unitCode': 'wmoUnit:degree_(angle)',
  'value': 164,
  'qualityControl': 'V'},
 'windSpeed': {'unitCode': 'wmoUnit:km_h-1',
  'value': 11.268,
  'qualityControl': 'V'},
 'windGust': {'unitCode': 'wmoUnit:km_h-1',
  'value': None,
  'qualityControl': 'Z'},
 'barometricPressure': {'unitCode': 'wmoUnit:Pa',
  'value': None,
  'qualityControl': 'Z'},
 'seaLevelPressure': {'unitCode': 'wmoUnit:Pa',
  'value': None,
  'qualityControl': 'Z'},
 'visi

In [69]:
rj['features'][0]['properties']['timestamp']

'2024-07-12T14:53:00+00:00'

In [70]:
rj['features'][-1]['properties']['timestamp']

'2024-07-05T15:53:00+00:00'

In [73]:
json.dumps({
                "station_id": station_id,
                "data_source": "weather.gov"
            })

'{"station_id": "TS654", "data_source": "weather.gov"}'

In [115]:
rj['features'][0]

{'id': 'https://api.weather.gov/stations/TS654/observations/2024-07-12T14:53:00+00:00',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-108.52, 40.38]},
 'properties': {'@id': 'https://api.weather.gov/stations/TS654/observations/2024-07-12T14:53:00+00:00',
  '@type': 'wx:ObservationStation',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 2048.3},
  'station': 'https://api.weather.gov/stations/TS654',
  'timestamp': '2024-07-12T14:53:00+00:00',
  'rawMessage': '',
  'textDescription': '',
  'icon': None,
  'presentWeather': [],
  'temperature': {'unitCode': 'wmoUnit:degC',
   'value': 23.89,
   'qualityControl': 'V'},
  'dewpoint': {'unitCode': 'wmoUnit:degC',
   'value': 0.23,
   'qualityControl': 'V'},
  'windDirection': {'unitCode': 'wmoUnit:degree_(angle)',
   'value': 164,
   'qualityControl': 'V'},
  'windSpeed': {'unitCode': 'wmoUnit:km_h-1',
   'value': 11.268,
   'qualityControl': 'V'},
  'windGust': {'unitCode': 'wmoUnit:km_h-1',
   'value': None,
   'qu

In [76]:
res = rc.get("weather_station:weather.gov:last_data_update_ts")

In [77]:
res

In [78]:
str(res)

'None'

In [79]:
rc_ts = rc.ts()

In [80]:
rc_ts.create("ExampleTSKey")

True

In [81]:
rc_ts.create("ExampleTSKey")

ResponseError: TSDB: key already exists

In [83]:
from time import time

In [84]:
rc_ts.add("ExampleTSKey2", int(time() * 1000), 1)

1720814963984

In [85]:
rc_ts.add("ExampleTSKey2", int(time() * 1000), 1)

1720814970139

In [86]:
rc_ts.get("ExampleTSKey2")

(1720814970139, 1.0)

In [90]:
rc_ts.range("ExampleTSKey2", "-", "+")

[(1720814963984, 1.0), (1720814970139, 1.0)]

In [88]:
pipe = rc_ts.pipeline()

In [89]:
pipe.add("ExampleTSKey2", int(time() * 1000), 1)

Pipeline<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

In [91]:
rc_ts.range("ExampleTSKey2", "-", "+")

[(1720814963984, 1.0), (1720814970139, 1.0)]

In [92]:
pipe.execute()

[1720815192159]

In [93]:
rc_ts.range("ExampleTSKey2", "-", "+")

[(1720814963984, 1.0), (1720814970139, 1.0), (1720815192159, 1.0)]

### Convert timestamp to unix ts

In [94]:
tsi = rj['features'][0]['properties']['timestamp']

In [95]:
tsi

'2024-07-12T14:53:00+00:00'

In [96]:
from datetime import datetime

In [97]:
datetime.strptime(tsi, "%Y-%m-%dT%H:%M:%S%z")

datetime.datetime(2024, 7, 12, 14, 53, tzinfo=datetime.timezone.utc)

In [98]:
datetime.strptime(tsi, "%Y-%m-%dT%H:%M:%S%z").timestamp()

1720795980.0

In [100]:
datetime.strptime('2024-07-12T14:53:00-05:00', "%Y-%m-%dT%H:%M:%S%z").timestamp()

1720813980.0

In [132]:
rc.llen("weather_station_process_queue")

12561

In [121]:
rc.delete("weather_station_process_queue")

1

In [122]:
rc.llen("weather_station_process_queue")

0

In [127]:
rc_ts.range("weather_station:weather.gov:787SE:data:temperature", "-", "+")

[(1720008000000, 13.2),
 (1720008600000, 13.06),
 (1720009200000, 12.97),
 (1720010400000, 12.77),
 (1720011000000, 12.69),
 (1720216200000, 30.28),
 (1720216800000, 30.11),
 (1720217400000, 29.94),
 (1720218000000, 29.67),
 (1720218600000, 29.17),
 (1720219200000, 28.94),
 (1720219800000, 28.67),
 (1720220400000, -99911999.0),
 (1720221000000, -99911999.0),
 (1720221600000, -99911999.0),
 (1720222800000, 26.63),
 (1720223400000, 26.64),
 (1720224600000, 26.14),
 (1720225200000, 26.29),
 (1720226400000, 26.11),
 (1720227000000, 26.12),
 (1720228200000, 26.38),
 (1720228800000, 26.6),
 (1720229400000, 26.72),
 (1720230000000, 26.83),
 (1720230600000, 26.94),
 (1720231800000, 27.17),
 (1720232400000, 27.0),
 (1720233600000, 26.55),
 (1720234200000, 26.15),
 (1720235400000, 25.32),
 (1720236000000, 24.97),
 (1720236600000, 24.66),
 (1720237200000, 24.33),
 (1720239000000, 22.92),
 (1720239600000, 22.22),
 (1720240200000, 21.47),
 (1720241400000, -99911999.0),
 (1720242000000, 20.17),
 (17